In [1]:
import requests
import pandas as pd
import json

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAGnaTwEAAAAAhRdM6yLmei6skyaWcjbx8IDFnlw%3DLPQHO2CTw1nVjjHLx3htgP9qmeCOgPpt96EdDujokNcWljI5iP'
headers = {'Authorization':('Bearer '+ bearer_token)}

n = 100                            # The total number of tweets we want
max_results = 10                  # The number of tweets to pull per request; must be between 10 and 100
total_retrieved = 0               # To keep track of when to stop
next_token = ""                   # Must be empty on first iteration
search_term = "covid"    # To form an advanced query, see here: https://twitter.com/search-advanced?lang=en
since_id = "1371600000000000000"  # The id of the oldest tweet you want to retrieve
df = pd.DataFrame(columns=['Retweets', 'Likes', 'Text'], index=['ID'])

# stop when we have n results
while total_retrieved < n:

  # the first time through the loop, we do not need the next_token parameter
  if next_token == "":
    # url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}'
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}'
  else:
    # url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}&next_token={next_token}'
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&next_token={next_token}'

  # These are the extra parameters we will add to the querystring; we won't store them all though; just want you to see what's possible
  url += f'&user.fields=created_at,description,entities,id,location,name,profile_image_url,protected,public_metrics,url,username,verified,withheld'
  url += f'&tweet.fields=attachments,author_id,context_annotations,conversation_id,created_at,entities,geo,id,in_reply_to_user_id,lang,possibly_sensitive,public_metrics,referenced_tweets,reply_settings,source,text,withheld'
  url += f'&expansions=attachments.poll_ids,attachments.media_keys,author_id,geo.place_id,in_reply_to_user_id,referenced_tweets.id,entities.mentions.username,referenced_tweets.id.author_id'
  url += f'&media.fields=duration_ms,height,media_key,preview_image_url,public_metrics,type,url,width'
  url += f'&place.fields=contained_within,country,country_code,full_name,geo,id,name,place_type'
  url += f'&poll.fields=duration_minutes,end_datetime,id,options,voting_status'

  # make the request to the Twitter API Recent Search endpoint
  response = requests.request("GET", url, headers=headers)
  json_data = json.loads(response.text)

  # Create the empty DataFrame with the columns you want
  df = pd.DataFrame(columns=['id', 'retweets', 'likes', 'url', 'text'])
  df.set_index('id', inplace=True)

  for tweet in json_data['data']:
    media_key = ""  # Reset to empty each time through the loop so that we can use it for a condition later

    # Store the data into variables
    tweet_id = tweet['id']
    retweet_count = tweet['public_metrics']['retweet_count']
    like_count = tweet['public_metrics']['like_count']
    image_url = ""
    text = tweet['text']

    # Find out if there is media
    if 'attachments' in tweet:
      if 'media_keys' in tweet['attachments']:
        media_key = tweet['attachments']['media_keys'][0]

    # If there is a media key in this tweet, iterate through tweet['includes']['media'] until we find it
    if media_key != "":
      for media in json_data['includes']['media']:
        if media['media_key'] == media_key: # Only if the media_key matches the one we stored
          if media['type'] == 'photo':      # Only if it is a photo; ignore videos
            image_url = media['url']        # Store the url in a variable

    # Add the new data to a new record in the DataFrame
    df.loc[tweet_id] = [retweet_count, like_count, image_url, text]

  # keep track of how many results have been obtained so far:
  # # total_retrieved += int(json_data['meta']['result_count'])
  total_retrieved += 1
  # try:
  #   next_token = json_data['next']['next_token']
  # except:
  #   print("No more results")

    # keep track of where to start next time, but quit if there are no more results
  try:
    next_token = json_data['meta']['next_token']
  except:
    break  

print(f'Number of records:\t{len(df)}')
df.head()

  

Number of records:	10


,retweets,likes,url,text
id,,,,
1441114616591904770,50,0,,RT @CarmenMForman: After the OKC Thunder annou...
1441114616495411202,0,1,,@Nate_Tice @TheFalcoholic Did anyone else noti...
1441114616461869057,12,0,,RT @tucamaiacaetano: - A chance da Terra ser a...
1441114616185032709,289,0,,RT @JamesMelville: As a consequence of the Cov...
1441114616017219586,929,0,,RT @NielsErikJ: Today I had to tell 6 people t...


In [ ]:
df.shape

In [ ]:
# In this example, only those tweets with photos/images are stored

n = 200                           # The total number of tweets we want
max_results = 100                 # The number of tweets to pull per request; must be between 10 and 100
total_retrieved = 0               # To keep track of when to stop
next_token = ""                   # Must be empty on first iteration
search_term = "canyoneering"             # To form an advanced query, see here: https://twitter.com/search-advanced?lang=en
since_id = "1371590000000000000"  # The id of the oldest tweet you want to retrieve

# Create the empty DataFrame with the columns you want
df_img = pd.DataFrame(columns=['id', 'retweets', 'likes', 'url', 'text'])
df_img.set_index('id', inplace=True)

# stop when we have n results
while total_retrieved < n:

  # the first time through the loop, we do not need the next_token parameter
  if next_token == "":
    # url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}'
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}'
  else:
    # url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}&next_token={next_token}'
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&next_token={next_token}'

  # These are the extra parameters we will add to the querystring; we won't store them all though; just want you to see what's possible
  url += f'&tweet.fields=attachments,public_metrics,text'
  url += f'&expansions=attachments.media_keys'
  url += f'&media.fields=media_key,type,url'

  # make the request to the Twitter API Recent Search endpoint
  response = requests.request("GET", url, headers=headers)
  try:  # Just in case we get an error
    json_data = json.loads(response.text)
  except:
    print(response.text)
  

  for tweet in json_data['data']:
    media_key = ""  # Reset to empty each time through the loop so that we can use it for a condition later

    # Store the data into variables
    tweet_id = tweet['id']
    retweet_count = tweet['public_metrics']['retweet_count']
    like_count = tweet['public_metrics']['like_count']
    image_url = ""
    text = tweet['text']

    # Find out if there is media
    if 'attachments' in tweet:
      if 'media_keys' in tweet['attachments']:
        media_key = tweet['attachments']['media_keys'][0]

    # If there is a media key in this tweet, iterate through tweet['includes']['media'] until we find it
    if media_key != "":
      for media in json_data['includes']['media']:
        if media['media_key'] == media_key: # Only if the media_key matches the one we stored
          if media['type'] == 'photo':      # Only if it is a photo; ignore videos
            image_url = media['url']        # Store the url in a variable
            
            # Only iterate if a photo is found
            total_retrieved += 1
            
            # Only add the record in the DataFrame if a photo is found
            df_img.loc[tweet_id] = [retweet_count, like_count, image_url, text]
            break

  # keep track of where to start next time, but quit if there are no more results
  try:
    next_token = json_data['meta']['next_token']
  except:
    break  

print(f'Number of records:\t{len(df_img)}')
# df_img.to_csv('twitter.csv')
df_img.head()

  

In [ ]:
# In this example, only those tweets with photos/images are stored

n = 200                           # The total number of tweets we want
max_results = 100                 # The number of tweets to pull per request; must be between 10 and 100
total_retrieved = 0               # To keep track of when to stop
next_token = ""                   # Must be empty on first iteration
search_term = "canyoneering"             # To form an advanced query, see here: https://twitter.com/search-advanced?lang=en
since_id = "1371590000000000000"  # The id of the oldest tweet you want to retrieve

# Create the empty DataFrame with the columns you want
df_img = pd.DataFrame(columns=['id', 'retweets', 'likes', 'text'])
df_img.set_index('id', inplace=True)

# stop when we have n results
while total_retrieved < n:
  print(total_retrieved)

  # the first time through the loop, we do not need the next_token parameter
  if next_token == "":
    # url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}'
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}'
  else:
    # url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&since_id={since_id}&next_token={next_token}'
    url = f'https://api.twitter.com/2/tweets/search/recent?query={search_term}&max_results={max_results}&next_token={next_token}'

  # These are the extra parameters we will add to the querystring; we won't store them all though; just want you to see what's possible
  url += f'&tweet.fields=attachments,public_metrics,text'
  url += f'&expansions=attachments.media_keys'
  url += f'&media.fields=media_key,type,url'

  # make the request to the Twitter API Recent Search endpoint
  response = requests.request("GET", url, headers=headers)
  try:  # Just in case we get an error
    json_data = json.loads(response.text)
  except:
    print(response.text)
  

  for tweet in json_data['data']:
    media_key = ""  # Reset to empty each time through the loop so that we can use it for a condition later

    # Store the data into variables
    tweet_id = tweet['id']
    retweet_count = tweet['public_metrics']['retweet_count']
    like_count = tweet['public_metrics']['like_count']
    image_url = ""
    text = tweet['text']

    # Find out if there is media
    if 'attachments' in tweet:
      if 'media_keys' in tweet['attachments']:
        media_key = tweet['attachments']['media_keys'][0]

    # If there is a media key in this tweet, iterate through tweet['includes']['media'] until we find it
    if media_key != "":
      for media in json_data['includes']['media']:
        if media['media_key'] == media_key: # Only if the media_key matches the one we stored
          if media['type'] == 'photo':      # Only if it is a photo; ignore videos
            image_url = media['url']        # Store the url in a variable
            
            

  df_img.loc[tweet_id] = [retweet_count, like_count, text]
  # keep track of where to start next time, but quit if there are no more results
  try:
    next_token = json_data['meta']['next_token']
    # Only iterate if a photo is found
    total_retrieved += 1
    # Only add the record in the DataFrame if a photo is found
    break
  except:
    break  

print(f'Number of records:\t{len(df_img)}')
# df_img.to_csv('twitter.csv')
df_img.head()

  

In [ ]:
df.shape

In [ ]:
df_img.shape